In [1]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
from keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from keras import backend as K
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
nltk.download('stopwords')
stop = stopwords.words('english')
tokenizer = Tokenizer(num_words = 8000, filters = '!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass


In [3]:
data_dev = []
data_test = []
data_train = []
file_name_dev = "audioDataDev.pickle"
file_name_test = "audioDataTest.pickle"
file_name_train = "audioDataTrain.pickle"

with (open(file_name_dev, "rb")) as f:
    while True:
        try:
            data_dev.append(pickle.load(f))
        except EOFError:
            break
f.close()

with (open(file_name_test, "rb")) as f:
    while True:
        try:
            data_test.append(pickle.load(f))
        except EOFError:
            break
f.close()

with (open(file_name_train, "rb")) as f:
    while True:
        try:
            data_train.append(pickle.load(f))
        except EOFError:
            break
f.close()

In [4]:
X_dev = []
y_dev = []
X_train = []
y_train = []
X_test = []
y_test = []

for i in range(len(data_dev)-1):
    X_dev.append(data_dev[i+1][4])
for i in range(len(data_dev)-1):
    y_dev.append(data_dev[i+1][6:12])
    
for i in range(len(data_test)-1):
    X_test.append(data_test[i+1][4])
for i in range(len(data_test)-1):
    y_test.append(data_test[i+1][6:12])
    
for i in range(len(data_train)-1):
    X_train.append(data_train[i+1][4])
for i in range(len(data_train)-1):
    y_train.append(data_train[i+1][6:12])

X_dev = np.array(X_dev)
y_dev = np.array(y_dev).astype(bool).astype(int)
X_test = np.array(X_test)
y_test = np.array(y_test).astype(bool).astype(int)
X_train = np.array(X_train)
y_train = np.array(y_train).astype(bool).astype(int)


In [5]:
def sent(X, y):
    data = []
    for i in range(1, len(X)):
        #add text
        temp = X[i].split('\n')
        text_obj = temp[0][6:]

        row = [text_obj, np.array(y[i])]
        data.append(row)

    df = pd.DataFrame(data, columns = ['text', 'sentiment'])
    return df

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def remove_stopwords(text):
    sw = stopwords.words('english')
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    return " ".join(text)

def sequences(dframe):
    tokenizer = Tokenizer(num_words = 8000, filters = '!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
    tokenizer.fit_on_texts(dframe['text'])
    X = tokenizer.texts_to_sequences(dframe['text'].values)
    y = dframe['sentiment'].values
    return X,y


In [6]:
dev_df = sent(X_dev, y_dev)
dev_df['text'] = dev_df['text'].apply(remove_punctuation).apply(remove_stopwords)
dev = sequences(dev_df)

test_df = sent(X_test, y_test)
test_df['text'] = test_df['text'].apply(remove_punctuation).apply(remove_stopwords)
test = sequences(test_df)

train_df = sent(X_train, y_train)
train_df['text'] = train_df['text'].apply(remove_punctuation).apply(remove_stopwords)
train = sequences(train_df)

In [7]:
X_dev_text = pad_sequences(dev[0])
y_dev_text = pad_sequences(dev[1])
X_test_text = pad_sequences(test[0])
y_test_text = pad_sequences(test[1])
X_train_text = pad_sequences(train[0])
y_train_text = pad_sequences(train[1])

In [8]:
X_train_text = X_train_text.reshape(X_train_text.shape[0], 1, X_train_text.shape[1])
X_test_text = X_test_text.reshape(X_test_text.shape[0], 1, X_test_text.shape[1])

In [9]:
X_train_text.shape

(16326, 1, 9)

In [10]:
#set up the LSTM model


#using the LSTM model with relu activation
adam = Adam(0.0002)
lstm = keras.Sequential()
lstm.add(LSTM(32, input_shape = (1, X_train_text.shape[2]), activation='relu', return_sequences=False))
lstm.add(Dense(6))
lstm.compile(loss='mean_squared_error',metrics='accuracy', optimizer=adam)

NameError: name 'Adam' is not defined

In [ ]:
y_train_text.shape

In [ ]:
lstm.fit(X_train_text, y_train_text, epochs=30, batch_size=10, verbose=1, shuffle=False)

loading testing set

In [ ]:
y_pred= lstm.predict(X_test_text)

In [ ]:
threshold = 0.41
yhat = np.where(y_pred > threshold, 1, 0)
print(y_pred[1])
print(y_test_text[1])

In [ ]:
results=[]
total = 0
similar = 0
for i in range(len(y_test_text)):
    for j in range(len(y_test_text[i])):
        if y_test_text[i][j] == y_pred[i][j]:
            similar += 1
        total += 1
acc = similar/total

print('>%.3f' % acc)
results.append(acc)

In [ ]:
y_test_f = []
y_hat_f = []
for i in range(len(y_pred[0])):
    y_temp_t = []
    y_temp_h = []
    for j in range(len(y_pred)):
        y_temp_t.append(y_test_text[j][i])
        y_temp_h.append(y_pred[j][i])
    y_test_f.append(y_temp_t)
    y_hat_f.append(y_temp_h)

In [ ]:
emotion = ['Happy', 'Sad', 'Anger', 'Surprise', 'Disgust', 'Fear']


for n, ax in zip(range(len(emotion)), axs.ravel()):
    # add a new subplot iteratively
    conf_matrix = confusion_matrix(y_true=y_test_f[n], y_pred=y_hat_f[n])
    #
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

    # chart formatting
    ax.set_title('${}$'.format(emotion[n]), fontsize=18)
    ax.set_xlabel('Predictions', fontsize=18)
    ax.set_ylabel('Actuals', fontsize=18)    

In [ ]:
from sklearn.metrics import mean_squared_error
print("y1 MSE:%.4f" % mean_squared_error(y_test_text[:,0], y_pred[:,0]))
print("y2 MSE:%.4f" % mean_squared_error(y_test_text[:,1], y_pred[:,1])) 
print("y3 MSE:%.4f" % mean_squared_error(y_test_text[:,2], y_pred[:,2])) 
print("y4 MSE:%.4f" % mean_squared_error(y_test_text[:,3], y_pred[:,3])) 
print("y5 MSE:%.4f" % mean_squared_error(y_test_text[:,4], y_pred[:,4])) 
print("y6 MSE:%.4f" % mean_squared_error(y_test_text[:,5], y_pred[:,5])) 

x_ax = range(len(X_test_text))
plt.title("LSTM multi-output prediction")
plt.scatter(x_ax, y_test_text[:,0],  s=6, label="y1-test")
plt.plot(x_ax, y_pred[:,0], label="y1-pred")
plt.scatter(x_ax, y_test_text[:,1],  s=6, label="y2-test")
plt.plot(x_ax, y_pred[:,1], label="y2-pred")
plt.legend()
plt.show()

In [ ]:
for n, ax in zip(range(len(emotion)), axs.ravel()):
    # add a new subplot iteratively
    conf_matrix = confusion_matrix(y_true=y_test_f[n], y_pred=y_hat_f[n])
    #
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

    # chart formatting
    ax.set_title('${}$'.format(emotion[n]), fontsize=18)
    ax.set_xlabel('Predictions', fontsize=18)
    ax.set_ylabel('Actuals', fontsize=18)    

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

range_vals = [i/100 for i in range(1, 30)]


precision = []
recall = []
accuracy = []
for i in range_vals:
    precision.append(precision_score(y_test_text[1], y_pred[1]))
    recall.append(recall_score(y_test_text[1], y_pred[1]))
    accuracy.append(accuracy_score(y_test_text[1], y_pred[1]))

In [ ]:
y_test_text[1]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize = (11, 7))
sns.lineplot(range_vals, precision)
sns.scatterplot(range_vals, precision)
sns.lineplot(range_vals, recall)
sns.scatterplot(range_vals, recall)
sns.lineplot(range_vals, accuracy)
sns.scatterplot(range_vals, accuracy)
plt.legend(['prec', 'rec', 'Acc'])
plt.show()